In [3]:
from refinery import RefineryData
import pandas as pd
import requests, warnings
import seasonal_plot
from datetime import datetime as dt

In [4]:
def ag_get_data(query):
    warnings.simplefilter('ignore', requests.packages.urllib3.exceptions.InsecureRequestWarning)
    params = {"Database": "OIL", "Query": query}
    resp = requests.post('https://TST-QDEV-AP1.petroineos.local:5001/genericdata/Fetch/', json=params, verify=False)
    json_result = resp.json()
    df = pd.DataFrame(json_result)
    return df

In [65]:
cntry_qry = "select distinct COUNTRY from oil.refinery.RefineryPlant"
country_df = ag_get_data(cntry_qry)
country_list = country_df['COUNTRY'].to_list()

In [66]:
region_mapping = pd.read_csv('tar_region_mapping.csv')
region_mapping_dedup = region_mapping[['Refinery Name', 'My Region']].copy().drop_duplicates(keep='first')

## data for Ali

In [67]:
unit_qry = "select distinct unit_group from oil.Refinery.RefineryUnit"
unit_list = ag_get_data(unit_qry)['unit_group'].to_list()
# unit_list = ['FCC','HCK']

In [6]:
type_qry = "select distinct COUNTRY, PLANT_NAME, UNIT_NAME, UTYPE_DESC from oil.Refinery.RefineryUnit"
type_mapping = ag_get_data(type_qry)

### planned

In [68]:
df_list=[]
for unit in unit_list:
    planned = RefineryData().get_outage('COUNTRY', country_list, unit,'Planned')
    df_list.append(planned.T)


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '

KeyboardInterrupt: 

In [8]:
full = pd.concat(df_list)
full.fillna(0, inplace=True)

In [9]:
full_m = full.T.resample('MS').mean()

In [10]:
short = full_m.loc['2016':'2022'].T

In [11]:
len(short)

7176

In [12]:
unit_pln_mapped = short.merge(type_mapping, how='left',left_on=['COUNTRY', 'PLANT_NAME', 'UNIT_NAME'], right_on=['COUNTRY', 'PLANT_NAME','UNIT_NAME'])

In [13]:
len(unit_pln_mapped)

7178

In [14]:
ctryplan = unit_pln_mapped.groupby(['COUNTRY', 'UTYPE_DESC']).sum().reset_index().melt(id_vars = ['COUNTRY', 'UTYPE_DESC'], var_name='Date',value_name='Kb/d')

In [ ]:
ctryplan

In [15]:
ref_plan = unit_pln_mapped.groupby(['PLANT_NAME', 'UTYPE_DESC']).sum().reset_index()
len(ref_plan)

4722

In [16]:
reg_plan = ref_plan.merge(region_mapping_dedup, how='left', left_on='PLANT_NAME', right_on='Refinery Name').groupby(['My Region', 'UTYPE_DESC']).sum()
reg_flat = reg_plan.reset_index().melt(id_vars = ['My Region', 'UTYPE_DESC'], var_name='Date',value_name='Kb/d')

In [ ]:
reg_flat

In [ ]:
type_mapping[type_mapping.duplicated(keep=False)].sort_values(['PLANT_NAME','UNIT_NAME']).to_csv('duplicates.csv')

### unplanned

In [17]:
def transform_monthly(df):
    df.fillna(0, inplace=True)
    df_m = df.T.resample('MS').mean()
    short_df = df_m.loc['2016':'2023'].T
    return short_df

In [18]:
def map_to_unit_type(df):
    df_mapped = df.merge(type_mapping, how='left',left_on=['COUNTRY', 'PLANT_NAME', 'UNIT_NAME'], right_on=['COUNTRY', 'PLANT_NAME','UNIT_NAME'])
    return df_mapped

In [19]:
def map_to_country_and_flatten(df):
    df_ctry = df.groupby(['COUNTRY', 'UTYPE_DESC']).sum().reset_index().melt(id_vars = ['COUNTRY', 'UTYPE_DESC'], var_name='Date',value_name='Kb/d')
    return df_ctry

In [20]:
def map_region_flatten(df):
    category = df.groupby(['PLANT_NAME', 'UTYPE_DESC']).sum().reset_index()
    region_mapped = category.merge(region_mapping_dedup, how='left', left_on='PLANT_NAME', right_on='Refinery Name').groupby(['My Region', 'UTYPE_DESC']).sum()
    region_flattened = region_mapped.reset_index().melt(id_vars = ['My Region', 'UTYPE_DESC'], var_name='Date',value_name='Kb/d')
    return region_flattened

In [21]:
# full_unplanned = pd.concat(unplan_list)
# unplan_m = map_to_unit_type(transform_monthly(full_unplanned))
# ctry_unplan = map_to_country_and_flatten(unplan_m)
# reg_unplan = map_region_flatten(unplan_m)

In [22]:
## new approach

In [23]:
def regional_query(outage_type):
    tar_list = []
    for unit in unit_list:
        data = RefineryData().get_outage('COUNTRY', country_list, unit, outage_type).T
        x = map_region_flatten(map_to_unit_type(transform_monthly(data)))
        tar_list.append(x)
    return pd.concat(tar_list)

In [24]:
def country_query(outage_type):
    tar_list = []
    for unit in unit_list:
        data = RefineryData().get_outage('COUNTRY', country_list, unit, outage_type).T
        x = map_to_country_and_flatten(map_to_unit_type(transform_monthly(data)))
        tar_list.append(x)
    return pd.concat(tar_list)

In [25]:
reg_plan = regional_query('Planned')


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '

In [26]:
reg_uplan = regional_query('Unplanned')


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '

In [27]:
ctry_plan = country_query('Planned')


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '

In [28]:
ctry_uplan = country_query('Unplanned')


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '

### write to excel

In [ ]:
writer = pd.ExcelWriter('TARs.xlsx', engine='xlsxwriter')
unit_pln_mapped.to_excel(writer, sheet_name='PQ_RefUnit', index=False)
ctry_plan.to_excel(writer, sheet_name='PQ_CtryPlan', index=False)
reg_plan.to_excel(writer, sheet_name='PQ_RegPlan', index=False)
ctry_uplan.to_excel(writer, sheet_name='PQ_CtryUnp', index=False)
reg_uplan.to_excel(writer, sheet_name='PQ_RegUnp', index=False)
writer.save()

## new TAR table

In [29]:
def convert_table(df):
    df_new = df[df['UTYPE_DESC']=='Atmospheric Distillation'].copy()
    df_short = df_new[df_new['Date']>='2021-1-1'].copy()
    df_pivot = df_short.pivot(index='My Region', columns='Date', values='Kb/d')
    return df_pivot

In [30]:
def map_group(df):
    tar_region_map = pd.read_excel('final_region_map.xlsx')
    table = df.merge(tar_region_map, how='left', left_on='My Region', right_on='My Region')
    sum_table = table.groupby('tar_region').sum()
    sum_table.loc['World'] = sum_table.sum()
    return sum_table.astype(int)

In [31]:
regional_planned_table = map_group(convert_table(reg_plan))
regional_unplanned_table = map_group(convert_table(reg_uplan))

In [32]:
sort_list=['North America','C & S America','Europe','FSU','Africa','AG','Asia Pacific', 'World']

In [42]:
writer = pd.ExcelWriter('tar_table.xlsx', engine='xlsxwriter')
regional_planned_table.loc[sort_list].to_excel(writer, sheet_name='planned')
regional_unplanned_table.loc[sort_list].to_excel(writer, sheet_name='unplanned')
writer.save()

## full table

In [69]:
total = RefineryData().get_outage('COUNTRY', country_list, 'CDU','Total')


            SELECT t.EVENT_ID, t.UNIT_NAME, t.UNIT_ID, t.PLANT_ID, t.PLANT_NAME, COUNTRY, [CAP_OFFLINE]
                  ,StartDate, EndDate, [EVENT_TYPE], [START_DATE], [END_DATE], [U_CAPACITY]
              FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
              inner join (
                  SELECT MaxDate = max([PDate]), EVENT_ID, tu.UNIT_ID, COUNTRY
                  FROM [OIL].[Refinery].[RefineryMaintenanceBest] t
                  inner join (
                        select PLANT_ID, PLANT_NAME, COUNTRY
                        from [OIL].[Refinery].[RefineryPlant]
                        where COUNTRY in ('Afghanistan', 'Algeria', 'Angola', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia-Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Cote dIvoire', 'Croatia', 'Cuba', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Finland', '

In [70]:
total.head()

COUNTRY                  Afghanistan                            \
PLANT_NAME         Hairatan Refinery                             
UNIT_NAME  Crude Distillation Unit 1 Crude Distillation Unit 2   
1991-09-01                       0.0                       0.0   
1991-09-02                       0.0                       0.0   
1991-09-03                       0.0                       0.0   
1991-09-04                       0.0                       0.0   
1991-09-05                       0.0                       0.0   

COUNTRY                    Algeria                                         \
PLANT_NAME          Arzew Refinery                        Skikda Refinery   
UNIT_NAME  Crude Distillation Unit Crude Distillation Unit 1 (Topping 10)   
1991-09-01                     0.0                                    0.0   
1991-09-02                     0.0                                    0.0   
1991-09-03                     0.0                                    0.0   
1991-09-04                     0.0                                    0.0   
1991-09-05                     0.0                                    0.0   

COUNTRY                                                             \
PLANT_NAME                                        Algiers Refinery   
UNIT_NAME  Crude Distillation Unit 2 (Topping 11)     Topping Unit   
1991-09-01                                    0.0              0.0   
1991-09-02                                    0.0              0.0   
1991-09-03                                    0.0              0.0   
1991-09-04                                    0.0              0.0   
1991-09-05                                    0.0              0.0   

COUNTRY                                                                \
PLANT_NAME Hassi-Messaoud Nr 1 & 2 Refineries          Adrar Refinery   
UNIT_NAME           Crude Distillation Unit 2 Crude Distillation Unit   
1991-09-01                                0.0                     0.0   
1991-09-02                                0.0                     0.0   
1991-09-03                                0.0                     0.0   
1991-09-04                                0.0                     0.0   
1991-09-05                                0.0                     0.0   

COUNTRY                       Angola                             ...  \
PLANT_NAME           Luanda Refinery                             ...   
UNIT_NAME  Crude Distillation Unit-2 Crude Distillation Unit- 1  ...   
1991-09-01                       0.0                        0.0  ...   
1991-09-02                       0.0                        0.0  ...   
1991-09-03                       0.0                        0.0  ...   
1991-09-04                       0.0                        0.0  ...   
1991-09-05                       0.0                        0.0  ...   

COUNTRY         Venezuela                          \
PLANT_NAME Amuay Refinery                           
UNIT_NAME         Crude 4 Crude 5 Crude 2 Crude 3   
1991-09-01            0.0     0.0     0.0     0.0   
1991-09-02            0.0     0.0     0.0     0.0   
1991-09-03            0.0     0.0     0.0     0.0   
1991-09-04            0.0     0.0     0.0     0.0   
1991-09-05            0.0     0.0     0.0     0.0   

COUNTRY                             Vietnam                    \
PLANT_NAME               Dung Quat Refinery Nghi Son Refinery   
UNIT_NAME  Atmospheric Distillation Unit 11             Crude   
1991-09-01                              0.0               0.0   
1991-09-02                              0.0               0.0   
1991-09-03                              0.0               0.0   
1991-09-04                              0.0               0.0   
1991-09-05                              0.0               0.0   

COUNTRY                   Yemen                                \
PLANT_NAME Little Aden Refinery                Marib Refinery   
UNIT_NAME                 CDU 1 CDU 2 Crude Distillatio

In [71]:
total_m = total.resample('MS').mean().loc['2022-07-01':'2023-03-31'].T

In [72]:
refinery_m = total_m.groupby(['COUNTRY','PLANT_NAME']).sum()

In [73]:
len(refinery_m)

634

In [74]:
dedup = region_mapping.drop_duplicates(subset=['Refinery Name', 'My Region'], keep='first')
dedup[dedup.duplicated(subset='Refinery Name', keep=False)].sort_values('Refinery Name')

Region   Country     Tar_Region       Refinery Name My Region  \
27   South America  Colombia  C & S America  Cartagena Refinery     LatAm   
97   Mediterranean     Spain         Europe  Cartagena Refinery       Med   
43             III    U.S.A.  North America  El Dorado Refinery        P3   
248             II    U.S.A.  North America  El Dorado Refinery        P2   

     PlantID  
27   1033810  
97   1065976  
43   1012387  
248  1013966

In [75]:
refinery_table = refinery_m.merge(region_mapping[['Country','Refinery Name', 'My Region']].drop_duplicates(keep='first'), how='left', left_on=['COUNTRY','PLANT_NAME'], right_on=['Country','Refinery Name'])

In [76]:
nwe = refinery_table[refinery_table['My Region']=='NWE']
med = refinery_table[refinery_table['My Region']=='Med']

In [77]:
def clean_set_index(df):
    df_new = df.set_index(['Country', 'Refinery Name'])
    df_new.drop(columns=['My Region'], inplace=True)
    df_t = df_new.T
    df_t.index = pd.to_datetime(df_t.index).date
    return df_t.T.sort_index().astype(int)

In [78]:
clean_set_index(nwe).style.background_gradient(cmap='Reds')

In [79]:
clean_set_index(med).style.background_gradient(cmap='Reds')

## TAR table

In [ ]:
planned = RefineryData().get_outage('COUNTRY', country_list, 'CDU', 'Planned')

In [ ]:
unplanned = RefineryData().get_outage('COUNTRY', country_list, 'CDU', 'Unplanned', exclude_cause=['Economic'])

In [ ]:
planned_m =  planned.resample('MS').mean().groupby(axis=1, level=[0, 1]).sum()
unplanned_m = unplanned.resample('MS').mean().groupby(axis=1, level=[0, 1]).sum()

In [ ]:
plan_mapped = planned_m.T.merge(region_mapping[['Refinery Name', 'Tar_Region', 'My Region']], left_on=['PLANT_NAME'], right_on='Refinery Name')
unplan_mapped = unplanned_m.T.merge(region_mapping[['Refinery Name', 'Tar_Region', 'My Region']], left_on=['PLANT_NAME'], right_on='Refinery Name')

In [ ]:
regional_plan = plan_mapped.groupby('Tar_Region').sum()
regional_unplan = unplan_mapped.groupby('Tar_Region').sum()
regional_plan.loc['World'] = regional_plan.sum()
regional_unplan.loc['World'] = regional_unplan.sum()

In [ ]:
sort_list=['North America','C & S America','Europe','FSU','Africa','AG','Asia Pacific', 'World']

In [ ]:
# regional_plan.loc[sort_list].astype(int).to_csv('test.csv')

In [ ]:
regional_unplan

In [ ]:
import ag_email_utils

In [ ]:
def send_mail_with_embedded_images(self, from_, to_, subject, body, images=None, auto=True):

In [ ]:
email_sender = ag_email_utils.AgEmailHelper()
email_sender.send_mail_with_embedded_images('syed', 'syed.ahmad@petroineos.com', 'test', 'hello', None, False)
# email_sender.send_mail()

In [ ]:
ag_email_utils.AgEmailHelper.send_mail

## NWE/MED plots

In [ ]:
nwe_plan = plan_mapped.groupby('My Region').sum().T[['NWE', 'Med']]
# nwe_plan['month'] = nwe_plan.index.month

In [ ]:
nwe_plan.index.month

In [ ]:
subregional_plan = plan_mapped.groupby('My Region').sum().T.loc['2017':'2022']
subregional_unplan = unplan_mapped.groupby('My Region').sum().T.loc['2017':'2022']

In [ ]:
subregional_plan.loc[2017:2022]

In [ ]:
subregional_plan['NWE'].loc['2017':'2022']

In [ ]:
seasonal_plot.seasonal_plot(series = subregional_plan['NWE'].loc['2017':'2022'], cutoff_year=2020, freq='MS')

In [ ]:
subregional_plan['NWE'].loc['2017':'2022']

In [ ]:
subregional_plan.index

### WoW changes

In [ ]:
planned_last = refinery_data.get_outage('COUNTRY', country_list, 'CDU', 'Planned', pdate=dt(2022,8,15))

In [ ]:
changes = planned.loc['2017':'2023'] - planned_last['2017':'2023']

In [ ]:
changes[changes.values<0].resample('MS').mean().T

In [ ]:
seasonal_plot.seasonal_plot()

### ad hoc

In [ ]:
capacities = RefineryData().get_capacity('COUNTRY', country_list, 'CDU')

In [ ]:
capacities.sum(axis=1).loc['2022':'2024'].plot()